In [ ]:

### import all the dependencies 
import requests
import csv
import json
import pandas as pd
import requests, json
import re
import configparser

In [ ]:
## Define all variables 
configFile = "config.cfg"
config = configparser.ConfigParser()
config.read(configFile)

TOKEN= config.get('FullStory', 'Token')


HEADERS = {'Authorization': 'Basic {}'.format(TOKEN)}
START_DATE = 1536364800 # 2018-09-07 17:00:00
LIST_REQUEST_URL = 'https://export.fullstory.com/api/v1/export/list?start={}'
GET_REQUEST_URL = 'https://export.fullstory.com/api/v1/export/get?id={}'

In [ ]:
## Calling FullStory APIS to get the data of all website traffic for last 3 weeks starting form 09/08 - 09/30
with requests.session() as session:
    session.headers.update(HEADERS)
    # Getting the list of data exports from fullstory
    print('Getting the exports list from fullstory')
    exports_list_response = session.get(LIST_REQUEST_URL.format(START_DATE)).json()

    exports = exports_list_response["exports"]
    exports_data = []
    

    # Downloading the exports from fullstory
    print("Starting downlaod of the Fullstory data exports")
    counter = 1
    for export in exports:
        print("Starting download for the bundle {}".format(counter))
        export_data = session.get(GET_REQUEST_URL.format(str(export['Id']))).json()
        exports_data = exports_data + export_data
        counter = counter + 1
    print("Downloaded {} dumps with {} total records".format(counter, len(exports_data)))


In [ ]:
# Loading the exports data to Pandas DataFrame for further processing 
exports_df = pd.DataFrame(exports_data)
print('Shape of exports dataframe: {}'.format(exports_df.shape))

In [ ]:
## Writing the dataframe to CSV file to avoid API call before every program run 
exports_df.to_csv("../data/Maindata.csv")
print("Data Successfully loaded to local CSV")